In [1]:
from sklearn.cluster import KMeans
import pandas as pd  
import numpy as np  
from sklearn.model_selection import GridSearchCV
from pandas import Series,DataFrame
import matplotlib.pyplot as plt 
import lightgbm as LGB
%pwd
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
Bike = pd.read_csv("C:/BDSE26/df_0907.csv", sep=",")
X_choice_reindex = Bike.reindex(columns=[
    'sno', 'Hr', 'RH', 'MRT_Dist', 'School_Dist', 'tot', 'remaining_vehicles'
    'Temperature', 'Precp', 'MRT_Out_ppl', 'MRT_Dist', 'School_Dist',
    'rent_count'
])
ds = X_choice_reindex.values
X = ds[:, 0:11]
y = ds[:, 11]
y = y.astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,mean_absolute_percentage_error
#定義回歸模型評估誤差指標
def median_absolute_percentage_error(y_true,y_pred):
    return np.median(np.abs((y_pred-y_true)/y_true))
def regression_metrics(true,pred):
    print('回歸模型評估指標結果:')
    print('均方誤差【MSE】:', mean_squared_error(true, pred))
    print('均方根誤差【RMSE】:',np.sqrt(mean_squared_error(true,pred)))
    print('平均絕對誤差【MAE】:',mean_absolute_error(true,pred))
    print('絕對誤差中位數【MedianAE】:',median_absolute_error(true,pred))
    print('平均絕對百分比誤差【MAPE】:',mean_absolute_percentage_error(true,pred))
    print('絕對百分比誤差中位數【MedianAPE】:',median_absolute_percentage_error(true,pred))

In [ ]:
#建立LGB的格式數據
lgb_train = LGB.Dataset(X_train, y_train)
lgb_eval = LGB.Dataset(X_test, y_test, reference=lgb_train)
#定義超参数dict
params = {
        'n_estimators': 100,
            # 'task': 'train',
            # 'boosting_type': 'gbdt',
            # 'objective': i,
            # 'metric':k,
            # 'max_depth': 7,
        'num_leaves': 10,
            # 'learning_rate': 0.1,
            # 'feature_fraction': 0.8,
            # 'bagging_fraction': 0.8,
            # 'bagging_freq': 5,
        'verbose': -1,
        'n_jobs': 7
}

In [ ]:
#定義callback回調
callback=[LGB.early_stopping(stopping_rounds=10,verbose=True),
          LGB.log_evaluation(period=10,show_stdv=True)]

In [ ]:
# 訓練 train
m1 = LGB.train(params,lgb_train,num_boost_round=2000,
               valid_sets=[lgb_train,lgb_eval],callbacks=callback)
y_pred = m1.predict(X_test, num_iteration=m1.best_iteration)  # 如果在訓練期間啓用了早期停止，可以通過best_iteration方式從最佳迭代中獲得預測
# 評估模型
print('The rmse of prediction is:',
      mean_squared_error(y_test, y_pred)**0.5)  # 計算真實值和預測值之間的均方根誤差


Training until validation scores don't improve for 10 rounds
[10]	training's l2: 62.7137	valid_1's l2: 62.4379
[20]	training's l2: 50.7359	valid_1's l2: 50.4574
[30]	training's l2: 44.9958	valid_1's l2: 44.7735
[40]	training's l2: 42.0811	valid_1's l2: 41.89
[50]	training's l2: 40.3268	valid_1's l2: 40.1431
[60]	training's l2: 39.1442	valid_1's l2: 38.962
[70]	training's l2: 38.0535	valid_1's l2: 37.8801
[80]	training's l2: 37.0753	valid_1's l2: 36.9061
[90]	training's l2: 36.4234	valid_1's l2: 36.2584
[100]	training's l2: 35.7769	valid_1's l2: 35.6181
[110]	training's l2: 35.3232	valid_1's l2: 35.1737
[120]	training's l2: 34.7975	valid_1's l2: 34.6636
[130]	training's l2: 34.4587	valid_1's l2: 34.3329
[140]	training's l2: 34.1105	valid_1's l2: 33.9904
[150]	training's l2: 33.8777	valid_1's l2: 33.7621
[160]	training's l2: 33.5736	valid_1's l2: 33.4683
[170]	training's l2: 33.3018	valid_1's l2: 33.2003
[180]	training's l2: 33.1037	valid_1's l2: 33.0127
[190]	training's l2: 32.9342	vali

C:\Users\j8003\AppData\Local\Temp\ipykernel_3536\17971333.py:4: RuntimeWarning: divide by zero encountered in true_divide
  return np.median(np.abs((y_pred-y_true)/y_true))


In [ ]:
import time
objective=['regression_l2','regression_l1','quantile','poisson','mape']
metrics=['l2','mae','quantile','poisson','mape']
metrics_test_data=pd.DataFrame(columns=['objective','metric','MAPE','Median APE','MAE'])
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
              '開始目標函數與評估函數評估')
for i in objective:
    for k in metrics:
        size=metrics_test_data.size
        params = {
            'n_estimators': 100,
            # 'task': 'train',
            # 'boosting_type': 'gbdt',
            # 'objective': i,
            # 'metric':k,
            # 'max_depth': 7,
            'num_leaves': 10,
            # 'learning_rate': 0.1,
            # 'feature_fraction': 0.8,
            # 'bagging_fraction': 0.8,
            # 'bagging_freq': 5,
            'verbose': -1,
            'n_jobs': 7
        }
        callback=LGB.early_stopping(stopping_rounds=10,verbose=0)
        gbm = LGB.train(params,lgb_train,num_boost_round=2000,
                valid_sets=lgb_eval,callbacks=[callback])
        y_pred = gbm.predict(X_test)
        metrics_test_data.loc[size]=[i,k,mean_absolute_percentage_error(y_test,y_pred),
                                     median_absolute_percentage_error(y_test,y_pred),
                                     mean_absolute_error(y_test,y_pred)
                                    ]
        print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),i,'+',k,' 完成评估',
              ' best iteration is:',gbm.best_iteration)
# metrics_test_data.to_csv('../result/m1_objective_metric_test.csv',encoding='gbk')
metrics_test_data

2022-09-09 21:54:04 開始目標函數與評估函數評估


C:\Users\j8003\AppData\Local\Temp\ipykernel_3536\17971333.py:4: RuntimeWarning: divide by zero encountered in true_divide
  return np.median(np.abs((y_pred-y_true)/y_true))


2022-09-09 21:54:05 regression_l2 + l2  完成评估  best iteration is: 255


C:\Users\j8003\AppData\Local\Temp\ipykernel_3536\17971333.py:4: RuntimeWarning: divide by zero encountered in true_divide
  return np.median(np.abs((y_pred-y_true)/y_true))


2022-09-09 21:54:07 regression_l2 + mae  完成评估  best iteration is: 433


...

2022-09-09 21:54:44 mape + quantile  完成评估  best iteration is: 102


C:\Users\j8003\AppData\Local\Temp\ipykernel_3536\17971333.py:4: RuntimeWarning: divide by zero encountered in true_divide
  return np.median(np.abs((y_pred-y_true)/y_true))


2022-09-09 21:54:47 mape + poisson  完成评估  best iteration is: 166
2022-09-09 21:54:53 mape + mape  完成评估  best iteration is: 329


C:\Users\j8003\AppData\Local\Temp\ipykernel_3536\17971333.py:4: RuntimeWarning: divide by zero encountered in true_divide
  return np.median(np.abs((y_pred-y_true)/y_true))


,objective,metric,MAPE,Median APE,MAE
0,regression_l2,l2,5.191826e+14,0.348009,3.998285
5,regression_l2,mae,5.056246e+14,0.347896,3.967138
10,regression_l2,quantile,5.422502e+14,0.352242,4.043015
15,regression_l2,poisson,6.473402e+14,0.359287,4.163478
20,regression_l2,mape,5.247070e+14,0.348594,4.015231
25,regression_l1,l2,4.363330e+14,0.346794,3.951885
30,regression_l1,mae,4.361113e+14,0.346617,3.949242
35,regression_l1,quantile,4.486933e+14,0.351195,4.003731
40,regression_l1,poisson,4.672982e+14,0.355482,4.064111
45,regression_l1,mape,4.491210e+14,0.351295,4.006639


In [ ]:
#我們使用網格搜索進行調參，因sklearn中的GridSearchCV過程中的scoring未提供我們重點觀測的Median APE指標，所以需要自定義。
from sklearn.metrics import make_scorer
neg_median_absolute_percentage_error=make_scorer(median_absolute_percentage_error,greater_is_better=False)

In [ ]:
#開始gridsearch
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),'開始樹深度和葉子结點數GridSearch')
model_lgb = LGB.LGBMRegressor(objective='quantile',
                              metric='quantile',
                              learning_rate=0.1,
                              subsample = 0.8,
                              colsample_bytree = 0.8,
                              subsample_freq = 5)
#調max_depth、num_leaves
params_test={
    'max_depth': range(7,11,1), 
    'num_leaves':range(10,90,10)  
}
gsearch1 = GridSearchCV(estimator=model_lgb,
                        param_grid=params_test,
                        scoring=neg_median_absolute_percentage_error,
                        cv=5,
                        verbose=1,
                        n_jobs=-1)
gsearch1.fit(X, y)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),'完成樹深度和葉子结點數GridSearch')
print('Best parameters found by grid search are:', gsearch1.best_params_)

2022-09-09 21:56:56 開始樹深度和葉子结點數GridSearch
Fitting 5 folds for each of 32 candidates, totalling 160 fits
2022-09-09 21:57:46 完成樹深度和葉子结點數GridSearch
Best parameters found by grid search are: {'max_depth': 9, 'num_leaves': 80}


In [ ]:
#参數在XX附近
params_test={
    'num_leaves':range(70,90,1)
}
gsearch1 = GridSearchCV(estimator=model_lgb,
                        param_grid=params_test,
                        scoring=neg_median_absolute_percentage_error,
                        cv=5,
                        verbose=1,
                        n_jobs=-1)
gsearch1.fit(X, y)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),'完成葉子结點數GridSearch')
print('Best parameters found by grid search are:', gsearch1.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
2022-09-09 22:11:05 完成葉子结點數GridSearch
Best parameters found by grid search are: {'num_leaves': 49}


In [ ]:
#調min_child_samples、min_child_weight 葉子結點最小數據數量與最小hessian
params_test={
    'min_child_samples':[19,20,21,22,23,24,25],
    'min_child_weight':[0.001,0.002]
}
gsearch1 = GridSearchCV(estimator=model_lgb,
                        param_grid=params_test,
                        scoring=neg_median_absolute_percentage_error,
                        cv=5,
                        verbose=1,
                        n_jobs=-1)
gsearch1.fit(X, y)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),'葉子結點最小數據數量與最小hessian GridSearch')
print('Best parameters found by grid search are:', gsearch1.best_params_)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
2022-09-09 22:11:41 葉子結點最小數據數量與最小hessian，GridSearch
Best parameters found by grid search are: {'min_child_samples': 25, 'min_child_weight': 0.001}


In [ ]:
#特徵與樣本的隨機採樣率subsample、colsample_bytree
params_test={
    'subsample': [0.7,0.8,0.9],
    'colsample_bytree': [0.7,0.8,0.9]  
}
gsearch1 = GridSearchCV(estimator=model_lgb,
                        param_grid=params_test,
                        scoring=neg_median_absolute_percentage_error,
                        cv=5,
                        verbose=1,
                        n_jobs=-1)
gsearch1.fit(X, y)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),'完成特徵與樣本的隨機採樣率 GridSearch')
print('Best parameters found by grid search are:', gsearch1.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
2022-09-09 22:12:19 完成特徵與樣本的隨機採樣率，GridSearch
Best parameters found by grid search are: {'colsample_bytree': 0.9, 'subsample': 0.9}


In [ ]:
#正則化參數
params_test={
    'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3],
    'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3]  
}
gsearch1 = GridSearchCV(estimator=model_lgb,
                        param_grid=params_test,
                        scoring=neg_median_absolute_percentage_error,
                        cv=5,
                        verbose=1,
                        n_jobs=-1)
gsearch1.fit(X, y)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),'完成正則化參數GridSearch')
print('Best parameters found by grid search are:', gsearch1.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
2022-09-09 22:13:40 完成正則化參數GridSearch
Best parameters found by grid search are: {'reg_alpha': 0, 'reg_lambda': 0.03}


In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier(is_unbalance=True)
model.fit(X_train,y_train)
from sklearn.metrics import accuracy_score

pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
